<a href="https://colab.research.google.com/github/tayfununal/2-Hidden-Layer-Neural-Networks/blob/master/one_hiddne_layer_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playground-data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plygdata as pg
import json
import requests

In [ ]:
Datas = json.loads(requests.get("https://raw.githubusercontent.com/tayfununal/2-Hidden-Layer-Neural-Networks/master/produced_XOR_Datas.json").text)

In [ ]:
df = pd.DataFrame(Datas["1"])
df

In [ ]:
X = np.array(df[[0,1]]).T
Y = np.array(df[[2]]).T

In [ ]:
print("Shape of X_train:", X.shape,
      "\nShape of y_train:", Y.shape)

In [ ]:
def initialization_parameters(x, y, num_node, random_seeds_one, random_seeds_two):
  np.random.seed(random_seeds_one)
  W1 = np.random.randn(num_node * x.shape[0]).reshape(num_node, x.shape[0]) * 0.001
  b1 = np.zeros((num_node,1))

  np.random.seed(random_seeds_two)
  W2 = np.random.randn(y.shape[0],num_node) * 0.001
  b2 = np.zeros((y.shape[0],1))

  assert W1.shape == (num_node,x.shape[0])
  assert b1.shape == (num_node, 1)

  assert W2.shape == (y.shape[0], num_node)
  assert b2.shape == (y.shape[0], 1)

  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)

def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

def forward_prop(x,parameters):
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = np.dot(W1, x) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  assert (A2.shape == (1, x.shape[1]))
  cache = {
      'Z1' : Z1,
      'A1' : A1,
      'Z2' : Z2,
      'A2' : A2
  }
  return A2, cache

def backward_prop(x, y, parameters, cache, learning_rate = 0.1, lambd=0.1):
  m = y.shape[1]
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']

  A1 = cache['A1']
  A2 = cache['A2']
  
  # Calculations of backward propagation: dW1, db1, dW2, db2
  dZ2 = A2 - y
  dW2 = (1 / m) * np.dot(dZ2, A1.T) + (lambd / m) * W2
  db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)

  dZ1 = np.multiply(np.dot(W2.T, dZ2), reluDerivative(A1))
  dW1 = (1 / m) * np.dot(dZ1, x.T) + (lambd / m) * W1  
  db1 = (1 / m) * np.sum(dZ1 , axis = 1, keepdims = True)

  # Updating parameters
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2
  
  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def cross_entropy_cost(y, A2, parameters, lambd = 0.1):
  m = y.shape[1]

  W1 = parameters["W1"]
  W2 = parameters["W2"]
  
  cross_entropy = np.multiply(np.log(A2 + 1e-15), y) + np.multiply((1 - y), np.log(1 - A2 + 1e-15))
  L2_regularization_cost = (np.sum(np.square(W1)) + np.sum(np.square(W2))) * (lambd / (2 * m))

  cost = (- 1.0 / m) * np.sum(cross_entropy) + L2_regularization_cost

  # Squeezing to avoid unnecessary dimensions 
  cost = np.squeeze(cost) 
  return cost

In [ ]:
def nn_model(x, y, parameters, number_of_iter = 1000):

  A2, cache = forward_prop(x, parameters)
  cost = cross_entropy_cost(y, A2 ,parameters)

  for i in range(1,number_of_iter):
    parameters = backward_prop(x, y, parameters, cache, learning_rate=0.1, lambd=0.1)
    A2, cache = forward_prop(x, parameters)
    cost = cross_entropy_cost(y, A2, parameters)
    
    #cost_value[i+1] = cost
    """
    if (i+1) % 1000 == 0:
     cost_value[i+1] = cost
     print("%i."%(i+1),cost)
     """
  return cost, parameters

In [ ]:
np.random.seed(65468)
random_seeds = np.zeros((100,2))
for i in range(0,100):
  for j in range(0,2):
    random_seeds[i,j] = np.random.random_integers(1,100001)

In [ ]:
from collections import Counter
average_aHundred_cost = dict()
for i in range(0,100):
  average_twenty_cost = dict()
  for number_of_neuron in range(1,21):
    parameters = initialization_parameters(X, Y, number_of_neuron, int(random_seeds[i,0]), int(random_seeds[i,1]))
    cost, parameters = nn_model(X , Y, parameters, number_of_iter = 1000)
    average_twenty_cost['{}'.format(number_of_neuron)] = cost
  print(i)
  average_aHundred_cost =  Counter(average_aHundred_cost) + Counter(average_twenty_cost)
average_aHundred_cost

In [ ]:
aveterage_aHundred_cost_2kezCalistirma =Counter({'1': 56.40593127856665,
         '2': 40.79322982879353,
         '3': 26.633210519191504,
         '4': 16.665555845229314,
         '5': 9.783761191069855,
         '6': 7.441549625752841,
         '7': 5.1284504099055015,
         '8': 4.241026418040571,
         '9': 4.033889653859869,
         '10': 3.8314469043198676,
         '11': 3.7000557728816834,
         '12': 3.4203377265475994,
         '13': 3.2944519866046007,
         '14': 2.8446698274271585,
         '15': 2.8406606970802466,
         '16': 2.733472057774313,
         '17': 2.6321627637484037,
         '18': 2.569273628709427,
         '19': 2.567677240124163,
         '20': 2.5653711829760106})

In [ ]:
average_aHundred_cost =Counter({'1': 56.40593127856665,
         '2': 40.79322982879353,
         '3': 26.633210519191504,
         '4': 16.665555845229314,
         '5': 9.783761191069855,
         '6': 7.441549625752841,
         '7': 5.1284504099055015,
         '8': 4.241026418040571,
         '9': 4.033889653859869,
         '10': 3.8314469043198676,
         '11': 3.7000557728816834,
         '12': 3.4203377265475994,
         '13': 3.2944519866046007,
         '14': 2.8446698274271585,
         '15': 2.8406606970802466,
         '16': 2.733472057774313,
         '17': 2.6321627637484037,
         '18': 2.569273628709427,
         '19': 2.567677240124163,
         '20': 2.5653711829760106})
​


In [ ]:
average_aHundred_cost.items()

In [ ]:
for i in range(1,21):
  average_aHundred_cost['{}'.format(i)] = average_aHundred_cost['{}'.format(i)]*(1/100)

In [ ]:
import collections
a=collections.OrderedDict(average_aHundred_cost.items())
a